# Credit Repayment Behaviour Analysis

The goal to find out whether the factors such as the marital status and the number of children of the borrowers affect the repayment of the loan to the bank on time. The Bank X has provided the data on clients' creditworthiness.

The report will be considered when building a credit scoring of a potential client of the bank. A credit scoring is used to evaluate the risks of the potential borrowers defaulting on their debt obligations.

# Data Description

***Note:*** the data provided by the Bank X is about bank clients in Russia, thus some contents of the dataset might be in Russian language.

children : the number of children in the family
<br>days_employed: how long the customer has worked
<br>dob_years: the customer’s age
<br>education: the customer’s education level
<br>education_id: identifier for the customer’s education
<br>family_status: the customer’s marital status
<br>family_status_id: identifier for the customer’s marital status
<br>gender: the customer’s gender
<br>income_type: the customer’s income type
<br>debt: whether the client has ever defaulted on a loan
<br>total_income: monthly income
<br>purpose: reason for taking out a loan

## Step 1. Data quality assessment

In [781]:
# importing libraries
import pandas as pd 
from pymystem3 import Mystem # getting a stemmer/lemmatizer for words in Russian

m = Mystem()

In [782]:
# opening the data file
df = pd.read_csv('clients_data.csv')

In [783]:
# getting the first 5 table strings.
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [784]:
# looking at the general information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [785]:
# checking numeric values
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [786]:
# checking for at the NaN values
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Summary:** We have a table with the total of 21525 rows and 12 columns of clients' data. Data types include float64, int64, and object. 
Findings:

1. 'total_income', 'days_employed' each have 2174 NaN values. 
2. 'days_employed': has negative and very big values, such as '-18388.949901' and '401755.400475'. The data in this column doesn't make sense as it is meant to represent the number of days of the employment. It is necessary to request more information about this data.
3. 'children': the number of children '-1' and '20'
4. 'dob_years': some of the client's data indicate their age as '0' years
5. 'education', 'education_id': there are 15 unique values in 'education' and 5 in 'education_id'
6. 'gender': 3 unique values: female, male, and N/A

Possible reasons for missing values & errors in the data above: In this case, it might be the concealment of the personal information by the client (about the employment and income level). Another possibility is the data entry errors.

## Step 2. Data preprocessing

### Processing the data with errors

Now we need to process the column 'children' where we saw such improbable values as -1 and 20 (number of children):

In [787]:
# check the values
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Following the logic that 20 and -1 were data entry errors, we can replace them assuming that 20 is an error for 2 children, and -1 is an error for 0 

In [788]:
df['children'] = df['children'].mask(df['children']==20, 2) 
df['children'] = df['children'].mask(df['children']==(-1), 0) 

    # check 
print(df['children'].value_counts())

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Summary:** We have processed the data in 'children' column and now the minimum number of children is 0 and maximum is 5, which seems reasonable

### Processing the missing values

To process the missing values in the data without affecting its accuracy, we need to divide data in groups (where possible) and fill in the missing values within each group.

For example, 'total_income' has 2174 missing values but we also know that we have a calumn 'income_types' (with such values as 'employee', 'retired') that could help us identify their level of income, so we will group 'total_income' by 'income_types' and will find a median value in each group.

In [789]:
# grouping 'total_income' by 'income_type'
income_dict = dict(df.groupby('income_type')['total_income'].median())

In [790]:
# replacing the missing values in 'total_income' column with values from 'income_dict'
df['total_income'] = df['total_income'].fillna(df['income_type'].apply(lambda x: income_dict.get(x)))

In [791]:

# filling in the missing values in 'days_employed' column with median, but without the dictionary
# because number of days of employment does not depend on the other parameters in the dataset
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
     #Check:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Summary:** We've processed the missing values and can move on to the next step

### Data type replacement

In [792]:
# replacing data types for the total_income', 'days_employed' with .astype() method
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')
    # Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Summary:** We've replaced the 'float64' data type with 'int64' for convenience and clarity

### Processing the data duplicates

In [793]:
# checking the values in 'education'
df['education'].value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

* **Eng translation** 
<br>secondary education    13750
<br>masters degree          4718
<br>SECONDARY EDUCATION      772
<br>Secondary Education      711
<br>bachelor degree          668
<br>MASTERS DEGREE           274
<br>Masters Degree           268
<br>primary education        250
<br>Bachelor Degree           47
<br>BACHELOR DEGREE           29
<br>PRIMARY EDUCATION         17
<br>Primary Education         15
<br>academic degree            4
<br>ACADEMIC DEGREE            1
<br> Academic Degree            1

We can see that we have duplicates in this column that need to be processed.

In [794]:
# using  str.lower() method
df['education'] = df['education'].str.lower()

    # Check
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

* **Eng translation**
<br>secondary education    15233
<br>masters degree          5260
<br>bachelor degree          744
<br>primary education        282
<br>academic degree            6

**Summary:** by bringing the data in 'education' to lowercase, we now have 'education' with 'education_id' in the same row and duplicates are sorted

In [795]:
# counting teh remaining number of duplicates in the dataframe
print('Remaining duplicates:',df.duplicated().sum())

Remaining duplicates: 71


In [796]:
# dropping duplicates + check
df = df.drop_duplicates().reset_index(drop= True)
print('Remaining duplicates after dropping:',df.duplicated().sum())

Remaining duplicates after dropping: 0


**Summary:** We've dropped all the data duplicates.

### Stemming/Lemmatization

We need to stem (or lemmatize) a 'purpose' column. To see which method is better, let's check its unique values first:

In [797]:
# using .value_couns()
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

* **Eng translation:**
<p>wedding ceremony                                 791<br>
having a wedding                                 768<br>
to have a wedding                                765<br>
real estate transactions                         675<br>
buy commercial real estate                       661<br>
housing transactions                             652<br>
buying property for renting out                  651<br>
transactions with the residential real estate    650<br>
purchase of the house                            646<br>
housing                                          646<br>
purchase of the house for my family              638<br>
construction of own property                     635<br>
property                                         633<br>
transactions with my real estate                 627<br>
building a real estate                           624<br>
buy real estate                                  621<br>
purchase of my own house                         620<br>
building a property                              619<br>
property renovation                              607<br>
buy residential real estate                      606<br>
buying my own car                                505<br>
going to university                              496<br>
car                                              494<br>
second-hand car purchase                         486<br>
cars                                             478<br>
buying a second-hand car                         478<br>
to own a car                                     478<br>
to buy a car                                     471<br>
car purchase                                     461<br>
supplementary education                          460<br>
purchase of a car                                455<br>
university education                             452<br>
education                                        447<br>
to get asupplementary education                  446<br>
getting an education                             442<br>
profile education                                436<br>
getting higher education                         426<br>
to become educated                               408


Find a stem the 'purpose' column:

In [798]:
def stem_text(text):
    words = text.split() # split text
    new_text = ' '.join([russian_stemmer.stem(w) for w in words])
    return new_text

df['purpose'] = df['purpose'].apply(stem_text)

# check the values
df['purpose'].value_counts() 

свадьб                            791
на проведен свадьб                768
сыгра свадьб                      765
операц с недвижим                 675
покупк коммерческ недвижим        661
операц с жил                      652
покупк жил для сдач               651
операц с коммерческ недвижим      650
покупк жил                        646
жил                               646
покупк жил для сем                638
строительств собствен недвижим    635
недвижим                          633
операц со сво недвижим            627
строительств жил недвижим         624
покупк недвижим                   621
покупк сво жил                    620
строительств недвижим             619
ремонт жил                        607
покупк жил недвижим               606
на покупк сво автомобил           505
заня высш образован               496
автомобил                         495
сделк с подержа автомобил         486
сво автомобил                     478
на покупк подержа автомобил       478
автомоб     

* **Eng translation:**
<p>car                                             972<br>
wedding ceremony                                791<br>
having a wedding                                768<br>
to have a wedding                               765<br>
real estate transaction                         675<br>
buy commercial real estate                      661<br>
housing transaction                             652<br>
buying property for renting out                 651<br>
transaction with the residential real estate    650<br>
purchase of the house                           646<br>
housing                                         646<br>
purchase of the house for my family             638<br>
construction of own property                    635<br>
property                                        633<br>
transaction with my real estate                 627<br>
building a real estate                          624<br>
buy real estate                                 621<br>
purchase of my own house                        620<br>
building a property                             619<br>
property renovation                             607<br>
buy residential real estate                     606<br>
buying my own car                               505<br>
going to university                             496<br>
second-hand car purchase                        486<br>
buying a second-hand car                        478<br>
to own a car                                    478<br>
to buy a car                                    471<br>
car purchase                                    461<br>
supplementary education                         460<br>
purchase of a car                               455<br>
university education                            452<br>
education                                       447<br>
to get asupplementary education                 446<br>
getting an education                            442<br>
profile education                               436<br>
getting higher education                        426<br>
to become educated                              408</p>

**Summary:** We've completed the stemming and now we can use it to group our data

### Categorizing Data

#### Categorising data in 'purpose'

We have identified 4 big groups in a 'purpose'. Now we will group the data by assigning 'автомобиль'("car"), 'свадьба'("wedding"), 'недвижимость'("real estate"), 'образование'("education"):

In [799]:
def lemma_group(text):
    if russian_stemmer.stem('автомобиль') in text:
        return 'автомобиль'
    if russian_stemmer.stem('свадьба') in text:
        return 'свадьба'
    if russian_stemmer.stem('недвижимость') in text or russian_stemmer.stem('имущество') in text or russian_stemmer.stem('дом') in text or russian_stemmer.stem('жильё') in text:
        return 'недвижимость'
    if russian_stemmer.stem('университет') in text or russian_stemmer.stem('образование') in text:
        return 'образование'

df['purpose'] = df['purpose'].apply(lemma_group)

    # check
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Which is the most popular 'purpose' of credit in the bank?

In [800]:
df['purpose'].value_counts()

недвижимость    10811
образование      4013
автомобиль       3829
свадьба          2324
Name: purpose, dtype: int64

* **Eng translation:**
<p>real estate    10811<br>
car             4306<br>
education       4013<br>
wedding         2324</p>

As we can see, the most popular reason to take a credit is a 'real estate' with a total of 10811 number of values. This seems like a reasonable answer as the cost of the real estat is very high and is unafordable for majority of the population. The least popular reason for taking a bank credit is 'wedding', with only 2324 values.

#### Categorizing the data in 'family_status'

In [801]:
# check the values
df['family_status'].value_counts() 

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

* **Eng translation**
<p>family_status	family_status_id<br>
0	married	12339<br>
1	civil partnership	4151<br>
2	widow / widower	2810<br>
3	divorced	1195<br>
4	unmarried	959</p>

The results for 'family_status' and 'family_status_id' represent the same thing, so we can group them together:

In [802]:
# creating the dictionary for 'family_status', 'family_status_id', droping the duplicates
family_dict = df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(family_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


* **Eng translation**
<p>family_status	family_status_id<br>
0	married	0<br>
1	civil partnership	1<br>
2	widow / widower	2<br>
3	divorced	3<br>
4	unmarried	4</p>

Now we can safely drop 'family_status_id' column.

In [803]:
# dropping 'family_status_id' column
df.drop(columns=['family_status_id'],inplace=True)

# check the results
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   children       21454 non-null  int64 
 1   days_employed  21454 non-null  int32 
 2   dob_years      21454 non-null  int64 
 3   education      21454 non-null  object
 4   education_id   21454 non-null  int64 
 5   family_status  21454 non-null  object
 6   gender         21454 non-null  object
 7   income_type    21454 non-null  object
 8   debt           21454 non-null  int64 
 9   total_income   21454 non-null  int32 
 10  purpose        20977 non-null  object
dtypes: int32(2), int64(4), object(5)
memory usage: 1.6+ MB


#### Categorizing the data in 'total_income'

In [804]:
# using .describe() method to know min, 25 %, mean, 75 % and max income
df['total_income'].describe() 

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Divide the data by 4 main levels of income: 'низкий'(as for 'low'), 'средний' (as for 'middle'), 'выше среднего' (as for 'high'), 'высокий' (as for 'very high')

In [805]:
def income_level(income):
    if income < 1.076522e+05:
        return 'низкий'
    if income < 1.518870e+05:
        return 'средний'
    if income < 2.024170e+05:
        return 'выше среднего'
    else:
        return 'высокий'

# using .apply() method to process column
df['income_level'] = df['total_income'].apply(income_level) 

# check the values
df['income_level'].value_counts() 

средний          6574
низкий           5368
высокий          4909
выше среднего    4603
Name: income_level, dtype: int64

* **Eng translation**
<p>middle       5870<br>
low     5359<br>
high    5317<br>
very high       4908</p>

We divided people by income and now its clear that the clients with the income tipe of 'middle' take the most credits in this bank

#### Categorizing the data in "debt"

In [806]:
# check the values
df['debt'].value_counts() 

0    19713
1     1741
Name: debt, dtype: int64

1 here means that the client has defaulted on a loan.<br>
0 means that the client repaid a loan.

Divide the data by 2 main categories: 'выплачен вовремя'(as for 'paid on time') and 'не выплачен' (as for 'defaulted')

In [807]:
def debt_type(debt):
    if debt == 0:
        return 'выплачен вовремя'
    if debt == 1:
        return 'не выплачен'

# use apply() method to process column
df['debt_type'] = df['debt'].apply(debt_type) 

# check the values
df['debt_type'].value_counts()

выплачен вовремя    19713
не выплачен          1741
Name: debt_type, dtype: int64

* **Eng translation**
<p>paid on time      19713<br>
defaulted     1741</p>

Let's count the difference between "paid on time" and "defaulted" (%)

In [808]:
percentage = 1741 / 19713 * 100
print(round(percentage,2),'%')

8.83 %


We've made a more visual representation of debtors and on time paid. The difference between "paid on time" and "defaulted" is 8.83 %. So the most of the clients successfully  repay their loan on time.

#### Categorizing the data in 'education'

In [809]:
 # check the values
df['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

* **Eng translation**
<p>secondary education    15172<br>
masters degree          5250<br>
bachelor degree          744<br>
primary education        282<br>
academic degree            6</p>

'secondary education' and 'masters degree' are the most popular groups

If we stand back and look, the results of 'education' and 'education_id' represent the samme thing
So we'll create dictionary for them, as we did for 'family_status_id'


In [810]:
# create the dictionary for 'education', 'education_id', drop duplicates
education_dict = df[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
display(education_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


* **Eng translation**
<p>education	education_id<br>
0	masters degree	0<br>
1	secondary education	1<br>
2	bachelor degree	2<br>
3	primary education	3<br>
4	academic degree	4</p>

Now we can safely we can drop "education_id" column.

In [811]:
df.drop(columns=['education_id'],inplace=True)

**Summary:** 
1. The most popular purpose for credit is for a real estate (10811 vs ~2000-4000 for other purposes)
2.  Majority of bank clients who apply for credit are married (12339 vs 959-4000 in other categories) 
3. Majority of people pay their debt on time (19713 vs 1741)
4. Most people who apply for the credit in the bank have the secondary education (15172 vs 5250 on masters degree and 6-700 on other types)

## Step 3. Answer the questions

**- Is there a relation between having kids and repaying a loan on time?**

Make the pivot table for analysis the "children" column. 

**Note:** A reminder that we divided the'debt' data by 2 main categories: 'выплачен вовремя'(as for 'paid on time') and 'не выплачен' (as for 'defaulted')

In [812]:
# create the pivot table 
credit_child = df.pivot_table(index=['children'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_child['debt_type_ratio'] = credit_child['не выплачен'] / credit_child['выплачен вовремя'] * 100
# sort by "debt_type_ratio"
credit_child.sort_values(by='debt_type_ratio', ascending=True)

debt_type,выплачен вовремя,не выплачен,debt_type_ratio
children,,,
0,13074.0,1064.0,8.138290
3,303.0,27.0,8.910891
1,4364.0,444.0,10.174152
2,1926.0,202.0,10.488058
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


In [813]:
# counting the difference
dif_per_children = credit_child['debt_type_ratio'].max() - credit_child['debt_type_ratio'].min()
print(round(dif_per_children,2),'%')

2.67 %


As we can see, ratio in 'debt_type_ratio' not that significant (only 2.67 %).

Divide people into two groups with and without children.

Сheck borrowers, who have children. And see how many people paid the loan on time.

In [814]:
debtor_parent = df[df['children'] > 0]['debt_type'].value_counts(normalize=True) # use value_counts() method
print(round(debtor_parent,2))

выплачен вовремя    0.91
не выплачен         0.09
Name: debt_type, dtype: float64


* **Eng translation**<br>
paid on time      0.91<br>
defaulted    0.09<br>

91 % of people with children repaid the loan on time.

Сheck borrowers who don't have children. And see how many people paid the loan on time.

In [815]:
debtor_childfree = df[df['children'] < 1]['debt_type'].value_counts(normalize=True) # use value_counts() method
print(round(debtor_childfree,2))

выплачен вовремя    0.92
не выплачен         0.08
Name: debt_type, dtype: float64


* **Eng translation**<br>
paid on time      0.92<br>
defaulted    0.08<br>

92% of people without children repaid the loan on time.

Count the difference:

In [816]:
dif_per_child = (debtor_childfree.loc['выплачен вовремя'] - debtor_parent.loc['выплачен вовремя']) * 100
print(round(dif_per_child,2),'%')

1.73 %


**Conclusion**

The difference between borrowers who have children and who haven't is only 1.73%. It is safe to assume it's not a signifficant value to take it into account. So there is not a (significant) relation between having children and repaying a loan on time.

**- Is there a relation between marital status and repaying a loan on time?**

In [817]:
# check the values 
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

* **Eng translation**
<p>family_status	family_status_id<br>
0	married	12339<br>
1	civil partnership	4151<br>
2	widow / widower	2810<br>
3	divorced	1195<br>
4	unmarried	959</p>

In [818]:
# create the pivot table 
credit_family = df.pivot_table(index=['family_status'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_family['debt_type_ratio'] = credit_family['не выплачен'] / credit_family['выплачен вовремя'] * 100
# sort by 'debt_type_ratio'
credit_family.sort_values(by='debt_type_ratio', ascending=True)

debt_type,выплачен вовремя,не выплачен,debt_type_ratio
family_status,,,
вдовец / вдова,896,63,7.031250
в разводе,1110,85,7.657658
женат / замужем,11408,931,8.160940
гражданский брак,3763,388,10.310922
Не женат / не замужем,2536,274,10.804416


In [819]:
dif_per_family = credit_family['debt_type_ratio'].max() - credit_family['debt_type_ratio'].min()
print(round(dif_per_family,2),'%')

3.77 %


**Conclusion**

The difference between borrowers who are maried and who are not is only 3.77%. It is safe to assume it's not a signifficant value to take it into account. So there is not a (significant) relation between having children and repaying a loan on time.

**- Is there a relation between income level and repaying a loan on time?**

Make the pivot table for analysis the 'income_level' colunm:

In [820]:
# create the pivot table 
credit_income = df.pivot_table(index=['income_level'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_income['debt_type_ratio'] = credit_income['не выплачен'] / credit_income['выплачен вовремя'] * 100
# sort by "debt_type_ratio"
credit_income.sort_values(by='debt_type_ratio', ascending=True)



debt_type,выплачен вовремя,не выплачен,debt_type_ratio
income_level,,,
высокий,4564,345,7.559159
низкий,4941,427,8.641975
выше среднего,4214,389,9.231134
средний,5994,580,9.676343


Count the difference

In [821]:
dif_per_income = credit_income['debt_type_ratio'].max() - credit_income['debt_type_ratio'].min()
print(round(dif_per_income,2),'%')

2.12 %


**Сonclusion**

There's no signifficant difference in 'dif_type_ratio' (oly 2.16 %). So there is no (significant) relation between income level and repaying a loan on time

**- How do different loan purposes affect on-time repayment of the loan?**

In [822]:
# create another the pivot table 
credit_purpose = df.pivot_table(index=['purpose'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_purpose['debt_type_ratio'] = credit_purpose['не выплачен'] / credit_purpose['выплачен вовремя'] * 100
# sort by "debt_type_ratio"
credit_purpose.sort_values(by='debt_type_ratio', ascending=True)

debt_type,выплачен вовремя,не выплачен,debt_type_ratio
purpose,,,
недвижимость,10029,782,7.797388
свадьба,2138,186,8.699719
образование,3643,370,10.156464
автомобиль,3470,359,10.345821


Count the difference

In [823]:
dif_per_purpose = credit_purpose['debt_type_ratio'].max() - credit_purpose['debt_type_ratio'].min()
print(round(dif_per_purpose,2),'%')

2.55 %


**Conclusion**

As we see, ratio in 'debt_type_purpose' not that signifficant (2.52 %). So loan purposes doesn't affect on-time repayment of the loan either.

## Step 4. General conclusion

In the process of analyzing the data, we did the following:

1. In Step 1 we got familiar with the provided dataset and set the plan for processing the data to analyze it later

2. In step 2 we filled all the missing values appropriately and got rid of the duplicates. Additionally, we used stemming method to process categorical data types. Then we moved on and categorized  the data where appropriate to beging analyzing data.

3. In step 3 we deeply analyzed the data and answered the questions that were interesting to the client (the Bank X)

***Summary***: Our main goal was to find out whether the customer’s marital status and number of children has an impact on them defaulting on a loan

*In conclusion*, we can say that customer’s marital status and number of children hasn't an impact on whether they will default on a loan. The difference between the 'defaulted' and 'paid on time' values isn't significant. And doesn't exceed 3.77% for marital status and 1.73% for the number of children.

And we do not see the connection between income, loan purposes and repayment of the loan on time.

Therefore, we can give loans more often not only to married, but also to other groups. And also to people with children. We need to look for ways to increase the return on credit in other categories.

**Recommendations**: 
1) Request for the correct data format in the 'days_employed' column and convert the data to the appropriate form

2) Deeper analysis of the data is required. There are many other factors that will help answer the question of whether the number of children or marital status affect the borrower's risk of defaulting. Specifically, it would be useful to add a few more detailed questions to the survey, e.g.: 
* Are there people with a disabilities/severe illnesses in the family?
* Are there any other debts in other banks/sources? 
* What are the average expenses per child in the family per month? etc.